<p><img src="https://files.cdn.thinkific.com/file_uploads/133292/images/a9e/c1a/bc3/P02.jpg"></p>

`MCSDSC02P01V3______`

`MCSDSC02P01V2______`

`MCSDSELIMC02P01V1010621`

## Постановка задачи

Без очереди — это мобильное приложение, которое помогает сделать заказ до прихода в кафе, чтобы не ждать официанта, приготовление еды и счет.
Необходимо изучить данные сервиса "Без очереди" с тестового запуска на территории дизайн-завода Флакон и Хлебозавода №9 и ответить на вопросы, поставленные менеджером проекта:

* На какие кафе приходится больше всего заказов? А на какие дни?
* Чаще ли приложением пользуются именно в обеденное время?
* Насколько большими компаниями  (или наоборот поодиночке) приходят в кафе?

## Импорт данных и библиотек

In [1]:
import pandas as pd

Прочитаем файл `bez_ocheredi.csv` и сохраним его в переменную `bo_data`.

In [2]:
bo_data =  pd.read_csv('https://raw.githubusercontent.com/cvetacea/bez_ocheredi_data/main/bez_ocheredi.csv')

Получим первые 10 строк таблицы.

In [3]:
bo_data.head(10)

,userID,ресторан,weekday,ordered,Столик для
0,b912c3c6a6ad,Iskra Bar,Monday,10:00,1
1,2dac0150717d,Iskra Bar,Monday,13:30,2
2,ba83aefab5dc,Iskra Bar,Tuesday,12:30,3
3,02996f1dd2ea,Iskra Bar,Wednesday,12:00,2
4,7c90be56d2ab,Iskra Bar,Thursday,14:30,3
5,51e0962387f7,Там Ем,Friday,12:30,2
6,c4f260a2b48a,NaN,Saturday,15:00,6
7,6b71309d6a8a,Крафтейнер,Monday,10:30,1
8,c5b969cb63a2,Бар Сибирь,Wednesday,15:30,2
9,b1570962e68c,Бар Сибирь,Wednesday,12:30,3


Получим общую информацию о данных из таблицы.

In [4]:
bo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   userID        1024 non-null   object
 1    ресторан     959 non-null    object
 2    weekday      1024 non-null   object
 3    ordered      1024 non-null   object
 4    Столик для   1024 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 40.1+ KB


Столбцы в `bo_data` содержат следующую информацию:

* ID пользователя приложения, сделавшего заказ
* название ресторана, в котором было сделан заказ
* день, время заказа
* на сколько человек был забронирован столик

### Выводы

* В таблице содержится 1024 строки и 5 столбцов
* 4 столбца (ID пользователя, название ресторана, день и время) содержат данные типа **object**, а столбец "Столик для" - данные типа **integer**
* Отсутствуют названия ресторанов в некоторых строках
* Общий объем памяти используемый базой данных 40.1 КB


## Предобработка данных

Получим названия столбцов.

In [5]:
bo_data.columns

Index(['userID ', ' ресторан ', ' weekday', ' ordered ', ' Столик для '], dtype='object')

Обращаем внимание, что помимо названий на разном языке, в них есть пробелы, которые затрудняют доступ к данным. Переименуем столбцы и проверим результат.

In [6]:
bo_data.rename(columns = {'userID ' : 'userID',' ресторан ' : 'restaurant',' weekday' : 'weekday', ' ordered ' : 'ordered', ' Столик для ' : 'guests'}, inplace = True)

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

ОК! А можно еще вот так было:
```python
bo_data.set_axis(['userID', 'restaurant', 'weekday', 'order_time', 'people_quantity'], axis='columns', inplace=True)
```

In [7]:
bo_data.columns

Index(['userID', 'restaurant', 'weekday', 'ordered', 'guests'], dtype='object')

Еще раз проверим данные на наличие пропусков, просуммировав их.

In [8]:
bo_data.isnull().sum()

userID         0
restaurant    65
weekday        0
ordered        0
guests         0
dtype: int64

Менеджер проекта предупреждал, что некоторые заказы заводились на тестовую карточку кафе без названия, поэтому есть смысл заполнить такой пропуск словом admin.

In [9]:
bo_data['restaurant'] = bo_data['restaurant'].fillna('admin')

Проверяем, что пропусков не осталось.

In [10]:
bo_data.isnull().sum()

userID        0
restaurant    0
weekday       0
ordered       0
guests        0
dtype: int64

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [11]:
bo_data.duplicated().sum()

24

In [12]:
bo_data = bo_data.drop_duplicates().reset_index(drop = True)

In [13]:
bo_data.duplicated().sum()

0

Скорее всего, такие полные дубликаты говорят о сбое при выгрузке данных. Необходимо сообщить об этом.

Еще одна проблема, о которой предупреждал менеджер проекта — дубликаты карточки Iskra Bar с неправильными названием бара - "Искра" и "Искр". На них заводились реальные заказы, поэтому необходимо вычислить их и переименовать на правильное название — Iskra Bar.

Объявим функцию `replace_restaurant()`, которая заменяет неправильное название, а потом проверяет успешность выполнения замены подсчетом (если замена прошла успешно — результат подсчета должен быть 0).

In [14]:
def replace_restaurant(data, bar_name, insert_name):
    data['restaurant'].replace(bar_name, insert_name, inplace = True)
    return data[data['restaurant'] == bar_name].count()

Передаем функции названия 'Искра' и 'Искр'.

In [15]:
replace_restaurant(bo_data, 'Искра', 'Iskra Bar')

userID        0
restaurant    0
weekday       0
ordered       0
guests        0
dtype: int64

In [16]:
replace_restaurant(bo_data, 'Искр', 'Iskra Bar')

userID        0
restaurant    0
weekday       0
ordered       0
guests        0
dtype: int64

### Выводы

Таблица подготовлена для совершения необходимых посчетов:
* Переименованы столбцы (удалены пробелы, название переведены в латиницу)
* Промаркированы данные, полученные при тестовом запуске
* Устранены дубликаты данных, связанных с багами работы приложения
* Объеденены по наименованию данные Iskra Bar

## Распределение заказов в течение недели

Считаем, на какие кафе приходится больше всего заказов.

In [17]:
bo_data['restaurant'].value_counts()

Iskra Bar     287
Там Ем        200
Бар Сибирь    159
Северия       107
Крафтейнер     78
Коворкафе      66
Азия           62
admin          41
Name: restaurant, dtype: int64

Группируем по `weekday` и считаем, на какие дни приходится больше всего заказов.

In [18]:
bo_data.groupby('weekday').count()

,userID,restaurant,ordered,guests
weekday,,,,
Friday,110,110,110,110
Monday,141,141,141,141
Saturday,52,52,52,52
Sunday,24,24,24,24
Thursday,180,180,180,180
Tuesday,251,251,251,251
Wednesday,242,242,242,242


<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Чтобы было красивее лучше вывести просто один столбец, т.к. у нас одинаковые значения по всем столбцам. И еще желательно сортировку сделать. Например так:

```python
bo_data.groupby('weekday')['restaurant'].count().sort_values(ascending=False)
```

Объявляем функцию, которая возвращает количество заказов для ресторана в определенный день.

In [19]:
def orders_by_weekday(weekday, restaurant):
    orders = bo_data[(bo_data['restaurant'] == restaurant) & (bo_data['weekday'] == weekday)]
    orders_count = orders['restaurant'].count()
    return orders_count

Передадим ей заведения Iskra Bar и Там Ем — два флагманских места, по которым просили результаты прежде всего — и понедельник, среду и пятницу в качестве дня недели.

In [20]:
#Iskra Bar, понедельник
orders_by_weekday('Monday', 'Iskra Bar')

48

In [21]:
#Iskra Bar, среда
orders_by_weekday('Wednesday', 'Iskra Bar')

53

In [22]:
#Iskra Bar, пятница
orders_by_weekday('Friday', 'Iskra Bar')

29

In [23]:
#Там Ем, понедельник
orders_by_weekday('Monday', 'Там Ем')

25

In [24]:
#Там Ем, среда
orders_by_weekday('Wednesday', 'Там Ем')

52

In [25]:
#Там Ем, пятница
orders_by_weekday('Fryday', 'Там Ем')

0

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Название дня с ошибкой, поэтому получилось ноль) Напиши Friday, там будет другое значение

Сведем результат в таблицу.

In [26]:
top_bo_data = [
    ['Monday','Iskra Bar', 48],
    ['Wednesday','Iskra Bar', 53],
    ['Friday','Iskra Bar', 29],
    ['Monday','Там Ем', 25],
    ['Wednesday','Там Ем', 52],
    ['Friday','Там Ем', 0],
]

columns = ['Weekday','Restaurant', 'Orders']

result = pd.DataFrame(data=top_bo_data, columns=columns)

result

,Weekday,Restaurant,Orders
0,Monday,Iskra Bar,48
1,Wednesday,Iskra Bar,53
2,Friday,Iskra Bar,29
3,Monday,Там Ем,25
4,Wednesday,Там Ем,52
5,Friday,Там Ем,0


### Выводы

* В **Iskra Bar** внедрение приложения прошло наиболее успешно, в этом ресторане было сделано больше всего заказов
* Исходя из суммарных значений всех заказов, посетители предпочитают посещать кафе в **будние дни**. Наиболее посещаемые дни - **вторник и среда**.  Исключение **Iskra Bar**, там наиболее посещаемые дни стали **понедельник и среда**.

## Распределение заказов в течение дня

Сравним количества заказов, сделанных в обеденное и вечернее время — с 12 до 16 и с 16 до 22. Для этого объявим функцию, которая принимает данные и 2 аргумента времени.

In [27]:
def time_orders(start_time, end_time):
    time = (bo_data['ordered'] >= start_time) & (bo_data['ordered'] < end_time)
    orders = bo_data[time]
    return orders['ordered'].count() 

Передадим промежуток с 12:00 и 16:00

In [28]:
time_orders('12:00', '16:00')

543

И с 16:00 до 22:00

In [29]:
time_orders('16:00', '22:00')

338

### Выводы

* Исходя из полученных данных, наибольший спрос на приложение в промежуток с 12:00 до 16:00 что может быть связано с желанием посетителей сэкономить время в обеденный перерыв на работе. 

## Типичное количество человек за столиком

Выясним, для какого количества человек обычно заказывают столик в разное время на примере Iskra Bar и Там Ем. Для этого напишем функцию, которая принимает данные, название ресторана, 2 времени и считает моду.

In [59]:
def moda_guests(input_rest, start_time, end_time):
    time = (bo_data['ordered'] >= start_time) & (bo_data['ordered'] < end_time)
    restaurant = bo_data['restaurant'] == input_rest
    new_data = bo_data[restaurant & time]
    moda = new_data['guests'].mode()
    return moda

Проверим функцию на двух ресторанах, а также дневном и вечернем промежутке времени.

In [60]:
moda_guests('Iskra Bar', '12:00', '16:00')

0    2
dtype: int64

In [62]:
moda_guests('Iskra Bar', '16:00', '22:00')

0    1
dtype: int64

In [63]:
moda_guests('Там Ем', '12:00', '16:00')

0    2
dtype: int64

In [64]:
moda_guests('Там Ем', '16:00', '22:00')

0    1
dtype: int64

### Выводы

* На примере данных из двух крупных заведений видно, что в обеденное время чаще бронируют столик на двоих, а в вечернее на одного. 

## Общие выводы

* В период запуска приложения больше всего заказов было сделано в сети ресторанов Iskra Bar.
* Приложением чаще всего пользуются в будние дни в обеденное время.
* Столики бронировали на одного или на двух человек.

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Проект принят! Очень хорошая работа!